In [2]:
import pandas as pd
LCO = False
t_vs_p_path = 'to_the_stars/true_vs_pred.csv' if LCO else 'to_the_universe_new/true_vs_pred.csv'
# Load and filter
results = pd.read_csv(t_vs_p_path, dtype={"drug": str, "cell_line": str, "CV_split": int}, index_col=0)
results = results[results["rand_setting"] == "predictions"]



In [3]:
# Ensure consistent types
results["drug"] = results["drug"].astype(str)
results["cell_line"] = results["cell_line"].astype(str)
results["CV_split"] = results["CV_split"].astype(int)

# Prepare naive predictions
naive = results[results["algorithm"] == "NaiveMeanEffectsPredictor"].copy()
naive["drug"] = naive["drug"].astype(str)
naive["cell_line"] = naive["cell_line"].astype(str)
naive["CV_split"] = naive["CV_split"].astype(int)

naive = naive[["drug", "cell_line", "CV_split", "y_pred"]].rename(columns={"y_pred": "naive_y_pred"})

# Merge
merged = results.merge(naive, on=["drug", "cell_line"], how="left")

# Drop any rows where naive_y_pred is missing (just in case)
merged = merged.dropna(subset=["naive_y_pred"])

# Subtract naive prediction from y_pred and y_true
merged["y_pred_normalized"] = merged["y_pred"] - merged["naive_y_pred"]
merged["y_true_normalized"] = merged["y_true"] - merged["naive_y_pred"]

merged

,model,drug,cell_line,y_true,y_pred,algorithm,rand_setting,LPO_LCO_LDO,CV_split_x,CV_split_y,naive_y_pred,y_pred_normalized,y_true_normalized
0,NaiveMeanEffectsPredictor_predictions_LPO_split_1,44185566,HEC-1-A,2.893982,2.877583,NaiveMeanEffectsPredictor,predictions,LPO,1,1.0,2.877583,0.000000,0.016399
1,NaiveMeanEffectsPredictor_predictions_LPO_split_1,9952709,DMS 273,2.373087,3.119245,NaiveMeanEffectsPredictor,predictions,LPO,1,1.0,3.119245,0.000000,-0.746158
2,NaiveMeanEffectsPredictor_predictions_LPO_split_1,9915743,BT 159,2.235896,1.185872,NaiveMeanEffectsPredictor,predictions,LPO,1,1.0,1.185872,0.000000,1.050024
3,NaiveMeanEffectsPredictor_predictions_LPO_split_1,216345,GMS-10,4.262503,3.991466,NaiveMeanEffectsPredictor,predictions,LPO,1,1.0,3.991466,0.000000,0.271037
4,NaiveMeanEffectsPredictor_predictions_LPO_split_1,16038120,YAPC,1.926562,1.547601,NaiveMeanEffectsPredictor,predictions,LPO,1,1.0,1.547601,0.000000,0.378961
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2792950,SuperFELTR_predictions_LPO_split_9,444795,ECC10,2.561087,0.873017,SuperFELTR,predictions,LPO,9,9.0,3.579793,-2.706775,-1.018705
2792951,SuperFELTR_predictions_LPO_split_9,444795,8505C,4.497569,1.259636,SuperFELTR,predictions,LPO,9,9.0,4.300310,-3.040674,0.197259
2792952,SuperFELTR_predictions_LPO_split_9,1285940,TE 441.T,4.276455,1.021108,SuperFELTR,predictions,LPO,9,9.0,3.991226,-2.970118,0.285229
2792953,SuperFELTR_predictions_LPO_split_9,1285940,HOP-62,3.199778,1.027243,SuperFELTR,predictions,LPO,9,9.0,4.534938,-3.507695,-1.335160
